# Analysis for Solar Energy / Solar Radiation by Location

## Theory: Can you beat rising energy prices with SOLAR

#### Goals 

- Gain insight on which areas will benefit from solar power over others
- Create an interactive display that a user can use to see how much they would benefit from solar power in their area and the price/power payoff.
- What areas of the planet is more prome to solar radiation, will this have a benefit for solar power?
- Moonlight coverage, looking at moonphases and the impact on solar power
- How many days of the year is it suitable for solar panels
- Estimate WATTS/pd based on conditions 

#### API Links

1. [Solcast](https://rapidapi.com/solcast/api/solcast/) - For Solar Radiation and Power
1. [DarkSky](https://rapidapi.com/darkskyapis/api/dark-sky/) - Advanced weather API for global coverage and historical data (Shutting down for Apple)
1. [VisualCrossing](https://www.visualcrossing.com/sign-up) - DarkSky Alternative

#### Notes

- Data is expensive!!
- Instead of pulling all postcodes in an area, get postcodes for center of all major cities.

- Based on this study [HumidityAndWin](https://www.solar.com/learn/how-do-wind-and-humidity-affect-solar-panel-efficiency/) solar panels are affected by humidity from water droplets, should be considered when cleaning data.

#### Columns of Note from VisualCrossing datasets

- Snowdepth
- UVIndex
- Sunrise/Sunset
- MoonPhase
- Conditions

#### Columns to drop

- Wind
- Temperature? (Need to research more, I don't think it has any affect)

#### Postcodes

- London (NW1 4NP)
- Leeds (LS1 1AA)
- Edinburgh (EH1 1AD)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Load Test Data

In [2]:
leeds_data_path = "test_datasets\Leeds_2021_Weather.csv"
london_data_path = "test_datasets\London_2021_Weather.csv"

df_leeds = pd.read_csv(leeds_data_path)
df_london = pd.read_csv(london_data_path)

FileNotFoundError: [Errno 2] No such file or directory: 'test_datasets\\Leeds_2021_Weather.csv'

In [ ]:
df_london.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,NW1 4NP,2021-01-01,4.9,-0.5,2.0,2.7,-3.6,-0.3,0.7,91.6,...,0.9,0,NaN,2021-01-01T08:06:27,2021-01-01T16:02:22,0.53,Rain,Clear conditions throughout the day with late ...,rain,"03769099999,03680099999,D5621,03672099999,0378..."
1,NW1 4NP,2021-01-02,5.1,1.4,3.7,2.9,-1.6,1.4,1.0,83.2,...,1.4,1,NaN,2021-01-02T08:06:19,2021-01-02T16:03:27,0.56,Rain,Clear conditions throughout the day with rain.,rain,"03680099999,D5621,03672099999,03781099999,0377..."
2,NW1 4NP,2021-01-03,5.9,1.1,3.7,5.6,-2.5,0.8,1.6,86.4,...,1.3,1,NaN,2021-01-03T08:06:07,2021-01-03T16:04:35,0.60,Rain,Clear conditions throughout the day with late ...,rain,"03680099999,D5621,03672099999,03781099999,0377..."
3,NW1 4NP,2021-01-04,5.5,3.4,4.2,4.0,-0.9,0.3,1.3,81.8,...,0.6,0,NaN,2021-01-04T08:05:52,2021-01-04T16:05:46,0.65,Rain,Clear conditions throughout the day with rain.,rain,"03680099999,D5621,03672099999,03781099999,0377..."
4,NW1 4NP,2021-01-05,4.6,2.4,3.6,0.7,-2.0,-0.5,1.0,82.7,...,0.8,1,NaN,2021-01-05T08:05:33,2021-01-05T16:06:59,0.70,Rain,Clear conditions throughout the day with rain.,rain,"03680099999,D5621,03672099999,03781099999,0377..."


In [ ]:
df_london['snow'].values

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
df_london.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [ ]:
df_london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              365 non-null    object 
 1   datetime          365 non-null    object 
 2   tempmax           365 non-null    float64
 3   tempmin           365 non-null    float64
 4   temp              365 non-null    float64
 5   feelslikemax      365 non-null    float64
 6   feelslikemin      365 non-null    float64
 7   feelslike         365 non-null    float64
 8   dew               365 non-null    float64
 9   humidity          365 non-null    float64
 10  precip            365 non-null    float64
 11  precipprob        365 non-null    int64  
 12  precipcover       365 non-null    float64
 13  preciptype        212 non-null    object 
 14  snow              0 non-null      float64
 15  snowdepth         9 non-null      float64
 16  windgust          139 non-null    float64
 1

## Data Cleaning

##### Notes

Cleaning:
- Dates for sunrise and sunset have an odd format, it might be best to split these or just keep the time.
- Snow column is empty in most cases dispite there been snow on the day, remove it.
- Remove columns that don't affect solar energy production. (SeaLevelPressure, description, Icon, Station)
- Rename 'name' to postcode since that will be the primary search condition

In [ ]:
def subset(cols, df):
    return df[[cols]]


In [ ]:
rel_cols = ['name', 'datetime', 'snowdepth', 'cloudcover', 'visibility', 'temp', 'solarradiation', 'solarenergy', 'uvindex']
subset_london = df_london[rel_cols]

In [ ]:
subset_london.head()

,name,datetime,snowdepth,cloudcover,visibility,temp,solarradiation,solarenergy,uvindex
0,NW1 4NP,2021-01-01,NaN,9.1,4.4,2.0,10.3,0.9,0
1,NW1 4NP,2021-01-02,NaN,8.9,24.7,3.7,15.2,1.4,1
2,NW1 4NP,2021-01-03,NaN,8.7,21.3,3.7,15.0,1.3,1
3,NW1 4NP,2021-01-04,NaN,8.5,30.2,4.2,7.7,0.6,0
4,NW1 4NP,2021-01-05,NaN,8.8,30.4,3.6,10.1,0.8,1


In [ ]:
df_london.shape

(365, 33)